# Initiation

In [51]:
import boto3
import json
import importlib
import os
import sys
import time
sys.path.append("../")

from dotenv import load_dotenv
from utils.openaiClient import OpenAIClient

# DEV AUTO RELOAD - REMOVE IN PRODUCTION
import textractParser
import textractClient
importlib.reload(textractClient)
importlib.reload(textractParser)

from textractClient import *
from textractParser import *

# Load the .env file (assuming it’s in the same folder)
load_dotenv()
AWS_REGION = os.getenv("AWS_REGION")

boto3.setup_default_session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=AWS_REGION
)

openai_client = OpenAIClient(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# CONFIGURATION

CATALOG_NAME = 'ms-02-02'
CATALOG_FILE = f'{CATALOG_NAME}.pdf'

S3_BUCKET = 'catalogs-for-extraction'
S3_CATALOG_PATH = f's3://{S3_BUCKET}/{CATALOG_FILE}'

TEXTRACT_OUTPUT_JSON_PATH = f"textract_outputs/textract_output_{CATALOG_NAME}.json"

# Run Textract Job

In [ ]:
# job_id = start_job(s3_bucket, document, features=['TABLES'], region=AWS_REGION)
job_id = "bce4b0bc30439528968cc52ca5ae225868056799096afbcc6944bdaef4fe81df"

status = is_job_complete(job_id, region=AWS_REGION)
results_pages = get_job_results(job_id, region=AWS_REGION)
save_results_to_file(results_pages, TEXTRACT_OUTPUT_JSON_PATH)

# Parse textract results

In [40]:
pages = load_textract_pages(TEXTRACT_OUTPUT_JSON_PATH)
all_blocks = convert_pages_to_blocks(pages)
id_map, type_map = build_block_maps(all_blocks)
table_blocks = type_map.get("TABLE", [])
print(f"Found {len(table_blocks)} tables in the document")

Found 4 tables in the document


In [44]:
tblock = table_blocks[2]

In [55]:
# PARSE TABLES
table_grid = convert_table_block_to_grid(tblock, id_map, replicate_data=True, header_scan_rows=2)

In [56]:
table_df = convert_grid_result_to_dataframe(table_grid)
table_df

,in. Main Line Tubing Size,Branch Swagelok Tube Fitting Size,Orifice in. (mm),Ordering Number,"Dimensions, in. (mm) / A","Dimensions, in. (mm) / B"
0,4 { 0.083,2 in.,1.500 (38.1),SS-D68TTE16S32,23.0 (584),17.2 (437)
1,4 { 0.083,1 1/2 in.,1.250 (31.8),SS-D67TTE16S24,23.0 (584),14.4 (365)
2,2 { 0.065,3/4 in.,0.516 (13.1),SS-D63TTE8S12,13.0 (330),8.19 (208)
3,2 { 0.065,1/2 in.,0.411 (10.4),SS-D63TTE8S8,13.0 (330),8.19 (208)
4,1 1/2 { 0.065,3/4 in.,0.516 (13.1),SS-D63TTE6S12,13.0 (330),7.13 (181)
5,1 1/2 { 0.065,1/2 in.,0.411 (10.4),SS-D63TTE6S8,13.0 (330),7.13 (181)
6,1 1/2 { 0.065,3/8 in.,0.281 (7.1),SS-D63TTE6S6,13.0 (330),6.94 (176)
7,1 { 0.065,3/4 in.,0.516 (13.1),SS-D63TTE4S12,13.0 (330),6.88 (175)
8,1 { 0.065,1/2 in.,0.411 (10.4),SS-D63TTE4S8,13.0 (330),6.88 (175)
9,1 { 0.065,3/8 in.,0.281 (7.1),SS-D63TTE4S6,13.0 (330),6.69 (170)
